In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import pandas as pd
import lightgbm as lgb
import numpy as np
from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, average_precision_score, auc
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from sklearn.model_selection import StratifiedKFold
import category_encoders
from catboost import Pool
from catboost.utils import get_roc_curve
from sklearn.metrics import auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import catboost

In [2]:
train_data = pd.read_csv('./train_dataset/train_public.csv')
test_public = pd.read_csv('./test_public.csv')
train_internet = pd.read_csv('./train_dataset/train_internet.csv', encoding='gb2312')

In [8]:
X_train = train_data.copy()
X_test = test_public.copy()
X_internet = train_internet.copy()

In [9]:
X_train = X_train.drop(['isDefault'], axis = 1)
X_train.shape

X_train['is_train'] = 1
X_test['is_train'] = 0

X_traintest = X_traintest = pd.concat([X_train, X_test], axis = 0)
X_traintest.shape

(15000, 39)

In [5]:
def time_format(x):
    #print(x)
    if x>= pd.to_datetime('2021-01-01'):
        t = '20' + str(x)[8:10] + '-' + str(x)[5:7] + '-01'
        #print('t=', t)
        return pd.to_datetime(t)
    return x

In [6]:
have_null_fea_dict = (X_traintest.isnull().sum()/len(X_traintest)).to_dict() 
fea_null_moreThanThreshold = {} 
for key,value in have_null_fea_dict.items(): 
    if value > 0.4: 
        fea_null_moreThanThreshold[key] = value
fea_null_moreThanThreshold

{}

In [10]:
unimport_fea = ['employer_type', 'industry', 'region', 'early_return', 'policy_code', 'pub_dero_bankrup', 'f1', 'del_in_18month', 'initial_list_status']
for cat in unimport_fea:
    X_internet = X_internet.drop(cat, axis=1)
    X_traintest = X_traintest.drop(cat, axis=1)

In [11]:
work_year_dict = {
    '< 1 year': 0,
    '1 year': 1,
    '2 years': 2,
    '3 years': 3,
    '4 years': 4,
    '5 years': 5,
    '6 years': 6,
    '7 years': 7,
    '8 years': 8,
    '9 years': 9,
    '10+ years': 10,
}


X_traintest['work_year'] = X_traintest['work_year'].map(work_year_dict)
X_internet['work_year'] = X_internet['work_year'].map(work_year_dict)

In [12]:
class_dict = {
    'A': 1,
    'B': 2,
    'C': 3,
    'D': 4,
    'E': 5,
    'F': 6,
    'G': 7,
}


X_traintest['class'] = X_traintest['class'].map(class_dict)
X_internet['class'] = X_internet['class'].map(class_dict)

## Date 

In [13]:
X_traintest['earlies_credit_mon'] = pd.to_datetime(X_traintest['earlies_credit_mon'])
X_internet['earlies_credit_mon'] = pd.to_datetime(X_internet['earlies_credit_mon'])

In [14]:
X_traintest['earlies_credit_mon'] = X_traintest['earlies_credit_mon'].apply(time_format)
X_internet['earlies_credit_mon'] = X_internet['earlies_credit_mon'].apply(time_format)

In [15]:
X_traintest['earlies_credit_mon_delta'] = (X_traintest['earlies_credit_mon'] - X_traintest['earlies_credit_mon'].min())  / np.timedelta64(1,'D')
X_internet['earlies_credit_mon_delta'] = (X_internet['earlies_credit_mon'] - X_internet['earlies_credit_mon'].min())  / np.timedelta64(1,'D')

In [18]:
X_traintest['issue_date'] = pd.to_datetime(X_traintest['issue_date'])
X_internet['issue_date'] = pd.to_datetime(X_internet['issue_date'])

In [19]:
X_traintest['issue_date'] = X_traintest['issue_date'].apply(time_format)
X_internet['issue_date'] = X_internet['issue_date'].apply(time_format)

In [20]:
X_traintest['issue_date_delta'] = (X_traintest['issue_date'] - X_traintest['issue_date'].min())  / np.timedelta64(1,'D')
X_internet['issue_date_delta'] = (X_internet['issue_date'] - X_internet['issue_date'].min())  / np.timedelta64(1,'D')

In [21]:
col_to_drop = ['issue_date', 'earlies_credit_mon']
X_traintest = X_traintest.drop(col_to_drop, axis=1)
X_internet = X_internet.drop(col_to_drop, axis=1)

## 交叉特征 

In [25]:
X_traintest['pro'] = X_traintest['interest'].mul(X_traintest['year_of_loan'])
X_traintest['loan_year'] = X_traintest['total_loan']/X_traintest['year_of_loan']
X_traintest['early_ratio'] = X_traintest['early_return_amount']/X_traintest['total_loan']
X_traintest['recircle_ratio'] = X_traintest['recircle_b']/X_traintest['total_loan']
X_traintest['recircle_amt'] = X_traintest['recircle_b'].mul(X_traintest['total_loan'])

In [26]:
X_internet['pro'] = X_internet['interest'].mul(X_internet['year_of_loan'])
X_internet['loan_year'] = X_internet['total_loan']/X_internet['year_of_loan']
X_internet['early_ratio'] = X_internet['early_return_amount']/X_internet['total_loan']
X_internet['recircle_ratio'] = X_internet['recircle_b']/X_internet['total_loan']
X_internet['recircle_amt'] = X_internet['recircle_b'].mul(X_internet['total_loan'])

## 特征补充 

In [27]:
numerical_fea = list(X_traintest.select_dtypes(exclude=['object']).columns) 
category_fea = list(filter(lambda x: x not in numerical_fea,list(X_traintest.columns)))

In [28]:
all_fea = X_traintest.columns
X_train = X_traintest[X_traintest.is_train == 1][all_fea]
y_train = train_data['isDefault']

X_test = X_traintest[X_traintest.is_train == 0][all_fea]

In [ ]:
X_train[category_fea] = X_train[category_fea].astype(str)
X_test[category_fea] = X_test[category_fea].astype(str)

X_train[numerical_fea] = X_train[numerical_fea].astype(float)
X_test[numerical_fea] = X_test[numerical_fea].astype(float)

In [30]:
model = catboost.CatBoostClassifier(cat_features = category_fea, random_state = 2080)
model.fit(X_train, y_train, verbose=400)

Learning rate set to 0.027538
0:	learn: 0.6612363	total: 160ms	remaining: 2m 39s
400:	learn: 0.2420175	total: 2.21s	remaining: 3.3s
800:	learn: 0.1937718	total: 4.08s	remaining: 1.01s
999:	learn: 0.1744778	total: 5.05s	remaining: 0us


In [31]:
feature_importances = model.get_feature_importance(prettified=True,
                                                   thread_count=-1,
                                                   verbose=False)

In [32]:
tr_cols = set(X_traintest.columns)
same_col = list(tr_cols.intersection(set(X_internet.columns)))

tr_cols.difference(set(same_col))

{'app_type', 'is_train', 'known_dero', 'known_outstanding_loan'}

In [33]:
for i in range(feature_importances['Feature Id'].size):
    if (feature_importances['Feature Id'][i] in same_col):
        if feature_importances['Importances'][i]>1:
            X_traintest[feature_importances['Feature Id'][i]+'_cnt'] = category_encoders.count.CountEncoder().fit_transform(X_traintest[feature_importances['Feature Id'][i]])
            X_internet[feature_importances['Feature Id'][i]+'_cnt'] = category_encoders.count.CountEncoder().fit_transform(X_internet[feature_importances['Feature Id'][i]])


## 训练、测试数据 

In [34]:
all_fea = X_traintest.columns
X_train = X_traintest[X_traintest.is_train == 1][all_fea]
X_train = pd.concat([X_train, train_data['isDefault']], axis=1)
y_train = train_data['isDefault']

X_test = X_traintest[X_traintest.is_train == 0][all_fea]

X_train = X_train.drop('is_train', axis=1)
X_test = X_test.drop('is_train', axis=1)

In [35]:
tr_cols = set(X_train.columns)
same_col = list(tr_cols.intersection(set(X_internet.columns)))
train_inteSame = X_internet[same_col].copy()

Inte_add_cos = list(tr_cols.difference(set(same_col)))
for col in Inte_add_cos:
    train_inteSame[col] = np.nan

## Stacking 

In [43]:
clf1 = lgb.LGBMClassifier(objective='binary',
                         boosting_type='gbdt',
                         tree_learner='serial',
                         num_leaves=32,
                         max_depth=7,
                         learning_rate=0.05,
                         n_estimators=10000,
                         subsample=0.8,
                         feature_fraction=0.6,
                         reg_alpha=0.5,
                         reg_lambda=0.5,
                         random_state=2021,
                         is_unbalance=True,
                         metric='auc',
                         verbose=-1)


clf2 = catboost.CatBoostClassifier(metric_period=400,
                                   cat_features = category_fea,
                                   random_state = 2080,
                                   eval_metric='AUC')

In [61]:
def train_model(data_, test_, y_, folds_, clf):
    oof_preds = np.zeros(data_.shape[0])
    sub_preds = np.zeros(test_.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in data_.columns if f not in ['loan_id', 'user_id', 'isDefault'] ]
    for n_fold, (trn_idx, val_idx) in enumerate(folds_.split(data_)):
        trn_x, trn_y = data_[feats].iloc[trn_idx], y_.iloc[trn_idx]
        val_x, val_y = data_[feats].iloc[val_idx], y_.iloc[val_idx]
        
        clf.fit(trn_x, trn_y, 
                eval_set= [(trn_x, trn_y), (val_x, val_y)], 
                verbose=400, early_stopping_rounds=50  #30
               )

        oof_preds[val_idx] = clf.predict_proba(val_x)[:, 1]
        sub_preds += clf.predict_proba(test_[feats])[:, 1] / folds_.n_splits
        
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(val_y, oof_preds[val_idx])))

        gc.collect()
        
    print('Full AUC score %.6f' % roc_auc_score(y_, oof_preds)) 
    
    test_['isDefault'] = sub_preds

    return oof_preds, test_[['loan_id', 'isDefault']], feature_importance_df

In [62]:
folds = KFold(n_splits=5, shuffle=True, random_state=546789)

In [63]:
oof_preds1, IntePre1, importances1 = train_model(X_train, train_inteSame, y_train, folds, clf1)
oof_preds2, IntePre2, importances2 = train_model(X_train, train_inteSame, y_train, folds, clf2)

[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[74]	training's auc: 0.946705	valid_1's auc: 0.893795
Fold  1 AUC : 0.893795
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[50]	training's auc: 0.929153	valid_1's auc: 0.893822
Fold  2 AUC : 0.893822
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[90]	training's auc: 0.958631	valid_1's auc: 0.87581
Fold  3 AUC : 0.875810
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature

Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.8886462767
bestIteration = 93

Shrink model to first 94 iterations.
Fold  1 AUC : 0.888646
Learning rate set to 0.052963
0:	test: 0.8595515	test1: 0.8654370	best: 0.8654370 (0)	total: 10.9ms	remaining: 10.9s


Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.8934020889
bestIteration = 83

Shrink model to first 84 iterations.
Fold  2 AUC : 0.893402
Learning rate set to 0.052963
0:	test: 0.8623572	test1: 0.8549997	best: 0.8549997 (0)	total: 35.4ms	remaining: 35.4s


Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.8739793023
bestIteration = 117

Shrink model to first 118 iterations.
Fold  3 AUC : 0.873979
Learning rate set to 0.052963
0:	test: 0.8678527	test1: 0.8376471	best: 0.8376471 (0)	total: 6.92ms	remaining: 6.91s


Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.86378455
bestIteration = 65

Shrink model to first 66 iterations.
Fold  4 AUC : 0.863785
Learning rate set to 0.052963
0:	test: 0.8620030	test1: 0.8554676	best: 0.8554676 (0)	total: 6.3ms	remaining: 6.29s


Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.8801357281
bestIteration = 90

Shrink model to first 91 iterations.
Fold  5 AUC : 0.880136
Full AUC score 0.879609


In [64]:
IntePre1['isDef'] = train_internet['is_default']
from sklearn.metrics import roc_auc_score
roc_auc_score(IntePre1['isDef'],IntePre1.isDefault)

E:\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0.7305715849000785

In [65]:
IntePre2['isDef'] = train_internet['is_default']
from sklearn.metrics import roc_auc_score
roc_auc_score(IntePre2['isDef'],IntePre2.isDefault)

E:\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0.735816662845307

In [66]:
IntePre = IntePre1.copy()
IntePre['isDefault'] = (IntePre['isDefault']+IntePre['isDefault'])/2

In [67]:
InteId = IntePre.loc[IntePre.isDefault<0.05, 'loan_id'].tolist()

X_train['dataSourse'] = 1
X_test['dataSourse'] = 1
train_inteSame['dataSourse'] = 0
train_inteSame['isDefault'] = X_internet['is_default']
use_te = train_inteSame[train_inteSame.loan_id.isin( InteId )].copy()
data = pd.concat([ X_train,X_test,use_te]).reset_index(drop=True)

In [98]:
train = data[data['isDefault'].notna()]
test  = data[data['isDefault'].isna()]




y = train['isDefault']
folds = KFold(n_splits=5, shuffle=True, random_state=1997)
oof_preds1, test_preds1, importances1 = train_model(train, test, y, folds, clf1)
oof_preds2, test_preds2, importances2 = train_model(train, test, y, folds, clf2)


[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[11]	training's auc: 0.982471	valid_1's auc: 0.978526
Fold  1 AUC : 0.978526
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[92]	training's auc: 0.99038	valid_1's auc: 0.97966
Fold  2 AUC : 0.979660
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[85]	training's auc: 0.989107	valid_1's auc: 0.979613
Fold  3 AUC : 0.979613
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_

E:\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Learning rate set to 0.079318
0:	test: 0.9646743	test1: 0.9634849	best: 0.9634849 (0)	total: 18.9ms	remaining: 18.9s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9791011807
bestIteration = 28

Shrink model to first 29 iterations.
Fold  1 AUC : 0.979101
Learning rate set to 0.079318


0:	test: 0.9642462	test1: 0.9642491	best: 0.9642491 (0)	total: 16.6ms	remaining: 16.6s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9804424637
bestIteration = 107

Shrink model to first 108 iterations.
Fold  2 AUC : 0.980442
Learning rate set to 0.079318
0:	test: 0.9643398	test1: 0.9634266	best: 0.9634266 (0)	total: 19.3ms	remaining: 19.3s


Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9801534581
bestIteration = 32

Shrink model to first 33 iterations.
Fold  3 AUC : 0.980153
Learning rate set to 0.079318


0:	test: 0.9653446	test1: 0.9599405	best: 0.9599405 (0)	total: 18.2ms	remaining: 18.2s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9799841995
bestIteration = 45

Shrink model to first 46 iterations.
Fold  4 AUC : 0.979984
Learning rate set to 0.079318
0:	test: 0.9631983	test1: 0.9686746	best: 0.9686746 (0)	total: 29.2ms	remaining: 29.1s


Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9816301483
bestIteration = 64

Shrink model to first 65 iterations.
Fold  5 AUC : 0.981630
Full AUC score 0.980007


E:\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [99]:
oof_preds1 = oof_preds1.reshape(-1, 1)
oof_preds2 = oof_preds2.reshape(-1, 1)


In [100]:
final_train = np.concatenate(( oof_preds1, oof_preds2), axis=1)
final_test = np.concatenate(( test_preds1[['isDefault']], test_preds2[['isDefault']]), axis=1)

In [101]:
final_test

array([[0.04489734, 0.0115659 ],
       [0.44312173, 0.04266063],
       [0.01565715, 0.00773683],
       ...,
       [0.00907311, 0.00856361],
       [0.02423823, 0.01561129],
       [0.01106783, 0.0058453 ]])

In [102]:
final_modoel = catboost.CatBoostClassifier(metric_period=400,
                                        random_state = 1998)

In [103]:
final_modoel.fit(final_train, y)

Learning rate set to 0.055355
0:	learn: 0.5132610	total: 6.1ms	remaining: 6.1s
400:	learn: 0.0554852	total: 2s	remaining: 2.98s
800:	learn: 0.0537918	total: 3.97s	remaining: 987ms
999:	learn: 0.0532551	total: 4.95s	remaining: 0us


In [104]:
predictions = final_modoel.predict_proba(final_test)[:,1]

In [105]:
predictions

array([0.08196992, 0.07735747, 0.00135032, ..., 0.00205789, 0.01055448,
       0.00084104])

In [106]:
submission = pd.read_csv('./submit_example.csv')
submission['isDefault'] = predictions
submission.to_csv('./submit/test_new_final.csv', index=False)